# Import Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

# Load Data

In [2]:
email_data = pd.read_csv("C:\\Users\\Swathi\\Desktop\\Excelr Classes\\Assignments\\Naive Baye-pending\\sms_raw_NB.csv",encoding = "ISO-8859-1")
email_data.head()

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or å£10,000..."
4,spam,okmail: Dear Dave this is your final notice to...


# Cleaning data 

- stopwords

- lower case

- numbers

- Spaces

In [3]:
import re
stop_words = []
with open("..\\Naive Baye-pending\\stop.txt") as f:
    stop_words = f.read()

In [4]:
# splitting the entire string by giving separator as "\n" to get list of 
# all stop words
stop_words = stop_words.split("\n")
# stop_words

In [5]:
def cleaning_text(i):
    i = re.sub("[^A-Za-z" "]+"," ",i).lower()
    i = re.sub("[0-9" "]+"," ",i)
    w = []
    for word in i.split(" "):
        if len(word)>3:
            w.append(word)
    return (" ".join(w))

In [6]:
email_data.text = email_data.text.apply(cleaning_text)
# email_data.text

In [7]:
# removing empty rows 
email_data.shape
email_data = email_data.loc[email_data.text != " ",:]

In [8]:
# creating a matrix of token counts for the entire text document 

def split_into_words(i):
    return [word for word in i.split(" ")]

# Splitting data into train and test data sets 

In [9]:
from sklearn.model_selection import train_test_split

email_train,email_test = train_test_split(email_data,test_size=0.3)

In [10]:
# Preparing email texts into word count matrix format 
emails_bow = CountVectorizer(analyzer=split_into_words).fit(email_data.text)
emails_bow

CountVectorizer(analyzer=<function split_into_words at 0x000000000968C828>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [11]:
# For all messages
all_emails_matrix = emails_bow.transform(email_data.text)
all_emails_matrix.shape # (5559,6661)
# For training messages
train_emails_matrix = emails_bow.transform(email_train.text)
train_emails_matrix.shape # (3891,6661)
# For testing messages
test_emails_matrix = emails_bow.transform(email_test.text)
test_emails_matrix.shape # (1668,6661)

(1668, 6661)

# Without TFIDF matrices 

In [12]:
# Preparing a naive bayes model on training data set 

from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.naive_bayes import GaussianNB as GB

In [13]:
# Multinomial Naive Bayes
classifier_mb = MB()
classifier_mb.fit(train_emails_matrix,email_train.type)
train_pred_m = classifier_mb.predict(train_emails_matrix)
accuracy_train_m = np.mean(train_pred_m==email_train.type) 
print("Train",accuracy_train_m*100)# 98%
test_pred_m = classifier_mb.predict(test_emails_matrix)
accuracy_test_m = np.mean(test_pred_m==email_test.type) 
print("Test",accuracy_test_m*100)# 97%

Train 98.71498329478283
Test 97.24220623501199


In [14]:
# Gaussian Naive Bayes 
classifier_gb = GB()
classifier_gb.fit(train_emails_matrix.toarray(),email_train.type.values) # we need to convert tfidf into array format which is compatible for gaussian naive bayes
train_pred_g = classifier_gb.predict(train_emails_matrix.toarray())
accuracy_train_g = np.mean(train_pred_g==email_train.type) 
print("Train",accuracy_train_g*100)# 91%

test_pred_g = classifier_gb.predict(test_emails_matrix.toarray())
accuracy_test_g = np.mean(test_pred_g==email_test.type) 
print("Test",accuracy_test_g*100)# 84%

Train 90.20817270624518
Test 83.15347721822542


# Learning Term weighting and normalizing on entire emails


In [15]:
tfidf_transformer = TfidfTransformer().fit(all_emails_matrix)
tfidf_transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [16]:
# Preparing TFIDF for train emails
train_tfidf = tfidf_transformer.transform(train_emails_matrix)

train_tfidf.shape # (3891, 6661)

(3891, 6661)

In [17]:
# Preparing TFIDF for test emails
test_tfidf = tfidf_transformer.transform(test_emails_matrix)

test_tfidf.shape #  (1668, 6661)

(1668, 6661)

In [18]:
# Preparing a naive bayes model on training data set 

from sklearn.naive_bayes import MultinomialNB as MB
from sklearn.naive_bayes import GaussianNB as GB

In [19]:
# Multinomial Naive Bayes
classifier_mb = MB()
classifier_mb.fit(train_tfidf,email_train.type)
train_pred_m = classifier_mb.predict(train_tfidf)
accuracy_train_m = np.mean(train_pred_m==email_train.type) 
print("Train",accuracy_train_m*100) # 96%

test_pred_m = classifier_mb.predict(test_tfidf)
accuracy_test_m = np.mean(test_pred_m==email_test.type) 
print("Test",accuracy_test_m*100)# 95%

Train 96.5047545618093
Test 95.80335731414868


In [20]:
# Gaussian Naive Bayes 
classifier_gb = GB()
classifier_gb.fit(train_tfidf.toarray(),email_train.type.values) # we need to convert tfidf into array format which is compatible for gaussian naive bayes
train_pred_g = classifier_gb.predict(train_tfidf.toarray())
accuracy_train_g = np.mean(train_pred_g==email_train.type) 
print("Train",accuracy_train_g*100)# 91%

test_pred_g = classifier_gb.predict(test_tfidf.toarray())
accuracy_test_g = np.mean(test_pred_g==email_test.type) 
print("Test",accuracy_test_g*100) # 84%

Train 90.20817270624518
Test 82.61390887290167
